In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import json
from dictor import dictor
from pprint import pprint
import os
import re

# specify the folder path
folder_path = Path('replay_jsons/')

def get_json_files(directory: str) -> dict:
    """
    Recursively traverse a directory tree and return a dictionary
    where the keys are the names of each subdirectory and the values
    are the names of the .json files in those subdirectories.

    Args:
        directory (str): The path to the directory to traverse.

    Returns:
        dict: A dictionary where the keys are subdirectory names and
        the values are lists of .json file names in those subdirectories.
    """
    # Initialize an empty dictionary to store the results
    result = {}

    # Create a Path object for the directory
    directory_path = Path(directory)

    # Traverse the directory tree recursively
    for subdirectory in directory_path.iterdir():
        # Check if the item is a directory
        if subdirectory.is_dir() and not subdirectory.name.startswith('.'):
            # Get the name of the subdirectory
            subdirectory_name = subdirectory.name

            # Initialize an empty list to store the JSON file names
            json_files = []

            # Traverse the subdirectory and find all JSON files
            for json_file in subdirectory.glob("[0-9]*.json"):
                if re.match(r'^\d+\.json$', os.path.split(json_file)[-1]):
                    # Add the name of the JSON file to the list
                    json_files.append(json_file)

            # Add the list of JSON file names to the result dictionary
            result[subdirectory_name] = json_files[0].resolve()
        elif subdirectory.name.endswith('.json'):
            result[subdirectory.name.split('.')[0]] = subdirectory.resolve()

    # Return the result dictionary
    return result


files = get_json_files(folder_path)
print(files.keys())

file_to_view = files['v225_opp_does_nothing']
print(f'Will be viewing:\n\t{file_to_view}')

with open(file_to_view, 'r') as f:
    full_replay = json.load(f)

seed = int(dictor(full_replay, 'configuration.seed'))
my_team = dictor(full_replay, 'info.TeamNames').index('Tim Child')
my_player = f'player_{my_team}'
replay_steps = full_replay['steps']
print(f'seed = {seed}\nmy team = {my_team}, my_player = {my_player}')

In [ ]:
from agent import Agent
from util import show_env, MyReplayEnv, CollectInfoFromEnv
from config import update_logging_level
import logging

In [ ]:
# Clear log file
LOGFILE = 'logfile.log'
with open(LOGFILE, 'w') as f:
    pass

info_collector = CollectInfoFromEnv(player=my_player)
myenv = MyReplayEnv(seed, Agent, full_replay, log_file_path=LOGFILE, my_player=my_player, collector=info_collector)
myenv.run_early_setup()
# myenv.show()

In [ ]:
update_logging_level(logging.WARNING)
# myenv.run_until_log_increases(200, max_step=100)
# myenv.show()

In [ ]:
import copy
# backup_env = copy.deepcopy(myenv)

In [ ]:
# myenv = copy.deepcopy(backup_env)
update_logging_level(logging.WARNING)
# myenv.run_to_step(36)
myenv.run_to_step(94)
myenv.show()

In [ ]:
fig = info_collector.plot_unit_last_update_step()
fig.update_layout(height=600, width=700)
fig.update_traces(mode='lines')
fig.show()
# pass

In [ ]:
update_logging_level(logging.DEBUG)
# myenv.run_to_step(8)
# myenv.undo()
# myenv.step()
myenv.step()
myenv.show()

In [ ]:
myenv.get_actions()

In [ ]:
myenv.env.state.units['player_0'].keys()
myenv.other_player

## Run one step at a time

In [ ]:
agent = myenv.agent
# unit = agent.master.units.get_unit('unit_9')
unit = agent.master.units.get_unit('unit_21')

In [ ]:
unit.status.last_action_update_step

In [ ]:
import numpy as np
from util import show_map_array, power_cost_of_path
import util

In [ ]:
connected_zeros = util.connected_array_values_from_pos(agent.master.maps.rubble, (19,36), 0)
other_lichen = agent.master.maps.lichen_strains
lichen = agent.master.game_state.board.lichen
figs = []
for arr in [connected_zeros, other_lichen, lichen]:
    figs.append(show_map_array(arr).update_layout(width=500, height=400))
fig = util.figures_to_subplots(figs).update_layout(width=900, height=700).update_yaxes(autorange='reversed')
fig.show()

In [ ]:
env.env.state.board.lichen.sum()

In [ ]:
np.sum((connected_zeros > 0) & (other_lichen < 0))

In [ ]:
# power_cost_of_path(path = np.array([[2,3], [2,4]]), agent.master.maps.rubble, 'LIGHT', 


In [ ]:
arr = env.env.state.board.lichen_strains[20:30,25:35]
arr2 = np.zeros_like(arr)
arr2[:5,5:] = 1
np.sum((arr & arr2 > 0))

In [ ]:
show_map_array(agent.master.game_state.board.lichen_strains)